# Atlas Prefilter

### Parameters

The next code block sets parameters that are used throughout the remainder of the notebook.

In [ ]:
# pylint: disable=invalid-name,missing-module-docstring

## Set atlas pre-filter parameters. This will eventually be derived from the SLURM input and config file paramters
## In a future version, both the template atlas and the RT adjustment atlases need to be retrieved from the MySQL database on NERSC

raw_data_dir = '/global/cfs/cdirs/metatlas/raw_data/jgi'
experiment = '20231103_JGI_MW_507961_Char_final-dil_EXP120B_C18_USDAY81385'

msms_refs_path = '/global/cfs/cdirs/metatlas/projects/spectral_libraries/20240222_labeled-addition_msms_refs.tab'
template_atlas_path = './metatlas-data/C18/C18_standards_positive.tsv'
rt_alignment_atlas_path = './rt_alignment_atlases/C18_rt_alignment_positive.tsv'

# either 'positive' or 'negative'
polarity = 'positive'

# filtering hit generation
ppm_tolerance = 5
extra_time = 0.5

# filtering hits
rt_window = 0.5
peak_height = 5e5
num_points = 4

msms_filter = True
msms_score = 0.65
msms_matches = 4
frag_tolerance = 0.02

# use regression for rt alignment, otherwise use median offset
rt_regression = False

# rt alignment model degree if using regression
model_degree = 1

In [ ]:
# pylint: disable=wrong-import-position,import-error,missing-class-docstring
parameters = {k: v for k, v in globals().items() if k[0] != "_" and k not in ["In", "Out", "get_ipython", "exit", "quit", "open"]}

import logging  # noqa: E402
from pathlib import Path  # noqa: E402


class StopExecution(Exception):
    def _render_traceback_(self):
        pass


logger = logging.getLogger("metatlas.jupyter")
kernel_def = """{"argv":["shifter","--entrypoint","--image=ghcr.io/biorack/metatlas/metatlas_shifter:latest","/usr/local/bin/python","-m",
                 "ipykernel_launcher","-f","{connection_file}"],"display_name": "Metatlas Targeted","language": "python",
                 "metadata": { "debugger": true }}"""
kernel_file_name = Path.home() / ".local" / "share" / "jupyter" / "kernels" / "metatlas-targeted" / "kernel.json"
try:
    has_root_kernel = Path("/root/.local/share/jupyter/kernels/papermill/kernel.json").is_file()
except PermissionError:
    has_root_kernel = False
if not has_root_kernel and not kernel_file_name.is_file():
    kernel_file_name.parent.mkdir(parents=True, exist_ok=True)
    with kernel_file_name.open(mode="w", encoding="utf-8") as f:
        f.writelines(kernel_def)
    logger.critical('CRITICAL: Notebook kernel has been installed. Set kernel to "Metatlas Targeted" and re-run notebook.')
    raise StopExecution
try:
    import atlas_prefilter
except ImportError as err:
    logger.critical('CRITICAL: Set notebook kernel to "Metatlas Targeted" and re-run notebook.')
    raise StopExecution from err

In [ ]:
atlas_prefilter.generate_outputs(
     raw_data_dir,
     experiment,
     msms_refs_path,
     template_atlas_path,
     rt_alignment_atlas_path,
     polarity,
     ppm_tolerance,
     extra_time,
     rt_window,
     peak_height,
     num_points,
     msms_filter,
     msms_score,
     msms_matches,
     frag_tolerance,
     rt_regression,
     model_degree
)